In [2]:
import librosa
import os
import numpy as np
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
from python_speech_features import fbank

function_1

In [3]:
def path_to_file(data_directory): 
    path_ids = []
    for folder in os.listdir(data_directory):
        if folder == "_background_noise_" :
            break
        k =  os.path.join(data_directory,folder)#join one or more path components
        for i in os.listdir(k):
            #print(k +"  "+folder+"  "+ i)
            if '.wav' in i:
                d = os.path.join(k , i)
                path_ids.append([d , folder , i])
    path_id_df = pd.DataFrame(path_ids,columns = ['path_to_file' , 'Speaker_id','data_set_id'])
    display(path_id_df)

In [4]:
k = path_to_file("E:\\mini project\\16000_pcm_speeches\\audio\\Train")
display(k)

,path_to_file,Speaker_id,data_set_id
0,E:\mini project\16000_pcm_speeches\audio\Train...,Benjamin_Netanyau,0.wav
1,E:\mini project\16000_pcm_speeches\audio\Train...,Benjamin_Netanyau,1.wav
2,E:\mini project\16000_pcm_speeches\audio\Train...,Benjamin_Netanyau,100.wav
3,E:\mini project\16000_pcm_speeches\audio\Train...,Benjamin_Netanyau,1000.wav
4,E:\mini project\16000_pcm_speeches\audio\Train...,Benjamin_Netanyau,1001.wav
...,...,...,...
6896,E:\mini project\16000_pcm_speeches\audio\Train...,Nelson_Mandela,995.wav
6897,E:\mini project\16000_pcm_speeches\audio\Train...,Nelson_Mandela,996.wav
6898,E:\mini project\16000_pcm_speeches\audio\Train...,Nelson_Mandela,997.wav
6899,E:\mini project\16000_pcm_speeches\audio\Train...,Nelson_Mandela,998.wav


None

Feature extraction (mfcc , logmelfilterbank) and store it in .pickle form

now we have to extract features from all audio files and prepare data frame

In [5]:
def features_extractor(file):
    
    #load the file audio
    audio, sample_rate = librosa.load(file,sr = 16000)
    
    #we extract mfcc
    mfcc_features = librosa.feature.mfcc(y=audio, sr=sample_rate,n_mfcc=40)
    mfcc_features = 20 * np.log10(np.maximum( mfcc_features,1e-5))
    #inorder to find out scaled feature we do mean of transpose of value
    feature = normalize_frames(mfcc_features, Scale=False)
   # mfcc_scaled_features = np.mean(mfcc_features.T, axis=0)
    return feature
    

In [6]:
#normalize_frames
def normalize_frames(m,Scale=True):
    if Scale:
        return (m - np.mean(m, axis=0)) / (np.std(m, axis=0) + 2e-12)
    else:
        return (m - np.mean(m, axis=0))
    
#log mel filter banks
def extract_filterbank_feature(filename):
    audio, sr = librosa.load(filename , sr = 16000 ,mono = True)
    filter_banks, energies = fbank(audio, samplerate=sr, nfilt=40, winlen=0.025)
    #f = filter_banks
    filter_banks = 20 * np.log10(np.maximum(filter_banks,1e-5))
    feature = normalize_frames(filter_banks, Scale=False)
    return feature

In [7]:
data_directory = "E:\\mini project\\16000_pcm_speeches\\audio\\Train"

Feature Extraction 

In [8]:
#now we iterate through audio files and extract features

# Using Mel_Frequency Cepstral Coefficients

mfcc_features = []
filter_bank_features = []

for folder in os.listdir(data_directory):
    k =  os.path.join(data_directory,folder)#join one or more path components
    for i in os.listdir(k):
        if '.wav' in i:
            #mfccs
            data = features_extractor(os.path.join(k , i))
            mfcc_features.append([data , folder])
            
            #log_mel_filter_bank
            data1 = extract_filterbank_feature(os.path.join(k , i))
            filter_bank_features.append([data1 , folder])
            
         
    

In [10]:
#converting extracted features to pandas dataframe
import pickle
mfcc_df = pd.DataFrame(mfcc_features,columns = ['mfcc_Features' , 'Class'])
#display(mfcc_df) #mfcc

filter_features_df = pd.DataFrame(filter_bank_features,columns = ['filter_bank_Features' , 'Class'])
#display(filter_features_df)

with open('featuremfcc.pickle','wb') as f:
    pickle.dump(mfcc_df,f)
    
with open('featurelogmelbank.pickle','wb') as f:
    pickle.dump(filter_features_df,f)

In [11]:
    
with open('featuremfcc.pickle','rb') as f:
    extracted_unpickle = pickle.load(f)
    
extracted_unpickle

,mfcc_Features,Class
0,"[[-99.34738, -98.47725, -97.23906, -33.294342,...",Benjamin_Netanyau
1,"[[-27.683777, -20.688728, -22.46952, -22.14833...",Benjamin_Netanyau
2,"[[-37.591957, 92.67914, 103.828964, 98.72943, ...",Benjamin_Netanyau
3,"[[-46.304546, 83.113464, 108.77193, 96.713394,...",Benjamin_Netanyau
4,"[[-27.386444, 101.812744, 102.052505, 96.54536...",Benjamin_Netanyau
...,...,...
6896,"[[-39.865807, -37.262108, -36.109642, -31.3023...",Nelson_Mandela
6897,"[[-43.887047, -39.662395, -46.20923, -34.82869...",Nelson_Mandela
6898,"[[-58.993683, -49.171284, -55.43467, -53.67086...",Nelson_Mandela
6899,"[[-26.568764, -31.756927, -37.970257, -39.0888...",Nelson_Mandela


In [12]:
with open('featurelogmelbank.pickle','rb') as f:
    extracted_unpickle = pickle.load(f)
extracted_unpickle

,filter_bank_Features,Class
0,"[[-0.3081066224772542, -2.359726114386291, -12...",Benjamin_Netanyau
1,"[[-17.44348629193614, -23.33493436472071, -38....",Benjamin_Netanyau
2,"[[-5.654847252898179, -4.103349722600186, -4.2...",Benjamin_Netanyau
3,"[[7.746209877459108, 8.238258552743048, 5.6769...",Benjamin_Netanyau
4,"[[-19.025099797366664, -18.707478966258833, -6...",Benjamin_Netanyau
...,...,...
6896,"[[-6.2559765799331615, 1.1037735413358831, 4.5...",Nelson_Mandela
6897,"[[-14.868866854903175, -20.412037581794962, -1...",Nelson_Mandela
6898,"[[-3.729822861490092, -0.735889405448134, 2.32...",Nelson_Mandela
6899,"[[21.33843501541223, 14.927778113420807, 17.23...",Nelson_Mandela
